In [1]:
import pandas as pd
import backtesting as bt
import pandas_ta as ta
import numpy as np
import datetime as dt
from IPython.display import clear_output
import breeze_client as bc

In [2]:
filename = 'data/NESIND/NESIND_1second_08-05-2024.csv'
data = bc.read_csv(filename)
data

,Close,exchange_code,High,Low,Open,stock_code,Volume
datetime,,,,,,,
2024-05-08 09:07:15,2502.45,NSE,2502.45,2502.45,2502.45,NESIND,0
2024-05-08 09:15:02,2502.60,NSE,2502.60,2502.60,2502.60,NESIND,1314
2024-05-08 09:15:13,2517.95,NSE,2517.95,2517.95,2517.95,NESIND,3382
2024-05-08 09:15:23,2514.95,NSE,2514.95,2514.95,2514.95,NESIND,5661
2024-05-08 09:15:33,2516.40,NSE,2516.40,2516.40,2516.40,NESIND,4991
...,...,...,...,...,...,...,...
2024-05-08 15:29:55,2539.50,NSE,2539.50,2539.50,2539.50,NESIND,0
2024-05-08 15:29:56,2539.50,NSE,2539.50,2539.50,2539.50,NESIND,0
2024-05-08 15:29:57,2539.00,NSE,2539.00,2539.00,2539.00,NESIND,4


In [10]:
data.reset_index(inplace=True)

In [4]:
def BBands_indicator(data,length,sd):
  bbands= ta.bbands(data.Close.s,length,sd)
  return bbands.to_numpy().T[:3]

def BBands_width(data,length,sd):
  bbands = ta.bbands(data.Close.s,length,sd)
  return bbands.to_numpy().T[3]

def BBands_per(data,length,sd):
  bbands = ta.bbands(data.Close.s,length,sd)
  return bbands.to_numpy().T[4]

In [5]:
class rsi(bt.Strategy):

    target = 9
    rsi_length = 14
    rsi_upperband = 70
    rsi_lowerband = 30
    bb_length = 20
    bb_sd = 1
    



    def init(self):
        self.rsi = self.I(ta.rsi,self.data.Close.s,self.rsi_length)
        self.vema = self.I(ta.ema,self.data.Volume.s,7)
        self.bbands = self.I(BBands_indicator,self.data,self.bb_length,self.bb_sd)

    
    def next(self):
        
        price = self.data.Close[-1]
        current_rsi = self.rsi[-1]

        vema = self.vema[-1]
        volume = self.data.Volume[-1]
        


        target= self.target/10
        target_precent = target/100
        target_price = (1-target_precent)*price


        

        if self.rsi_upperband <= current_rsi :#and vema < volume:

            if self.position.is_long:
                self.position.close()

            if not self.position.is_short:
                self.sell(size = 200)

        if self.rsi_lowerband >= current_rsi:

            if not self.position.is_long:
                self.buy(size=200)

            if self.position.is_short:
                self.position.close()
            


        
    

In [11]:
eng = bt.Backtest(data,rsi,cash=4000000,commission=0.002)

C:\Users\Ankit\AppData\Local\Temp\ipykernel_4808\188491853.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  eng = bt.Backtest(data,rsi,cash=4000000,commission=0.002)


In [12]:
stats = eng.run()
stats

Start                                     0.0
End                                   10415.0
Duration                              10415.0
Exposure Time [%]                   99.798387
Equity Final [$]                   3944267.28
Equity Peak [$]                    4000613.18
Return [%]                          -1.393318
Buy & Hold Return [%]                1.382645
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.408432
Avg. Drawdown [%]                   -0.188137
Max. Drawdown Duration                10359.0
Avg. Drawdown Duration               1298.875
# Trades                                 51.0
Win Rate [%]                              0.0
Best Trade [%]                      -0.032048
Worst Trade [%]                     -0.549506
Avg. Trade [%]                    

In [13]:
eng.plot()

C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1400', ...)

In [9]:
def optim_func(series):

    if series['Win Rate [%]'] > 65 and series['# Trades'] > 30:
        return series['Return [%]']
    
    


    else:
        return -1000000

opt_stats,heatmap = eng.optimize(
    rsi_length = range(5,22),
    rsi_upperband = range(45,85),
    
    maximize = optim_func,
    
    return_heatmap = True,
    max_tries = 1000

)
opt_stats

c:\Users\himan\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 680 configurations.
  output = _optimize_grid()
c:\Users\himan\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\himan\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\himan\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by

Start                     2024-01-29 09:05:00
End                       2024-04-26 15:25:00
Duration                     88 days 06:20:00
Exposure Time [%]                   83.380527
Equity Final [$]                   3165561.04
Equity Peak [$]                     4000000.0
Return [%]                         -20.860974
Buy & Hold Return [%]                0.517696
Return (Ann.) [%]                  -61.658674
Volatility (Ann.) [%]                1.702935
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -20.860974
Avg. Drawdown [%]                  -20.860974
Max. Drawdown Duration       88 days 05:30:00
Avg. Drawdown Duration       88 days 05:30:00
# Trades                                  838
Win Rate [%]                         15.75179
Best Trade [%]                       1.439869
Worst Trade [%]                     -1.020333
Avg. Trade [%]                    

In [61]:
opt_stats

Start                     2024-01-29 09:05:00
End                       2024-04-26 15:25:00
Duration                     88 days 06:20:00
Exposure Time [%]                   27.641037
Equity Final [$]                   4016567.24
Equity Peak [$]                    4028913.36
Return [%]                           0.414181
Buy & Hold Return [%]                0.517696
Return (Ann.) [%]                     1.72217
Volatility (Ann.) [%]                   1.465
Sharpe Ratio                         1.175543
Sortino Ratio                        1.844217
Calmar Ratio                          3.83626
Max. Drawdown [%]                   -0.448919
Avg. Drawdown [%]                   -0.080914
Max. Drawdown Duration       38 days 01:25:00
Avg. Drawdown Duration        2 days 00:38:00
# Trades                                   19
Win Rate [%]                        57.894737
Best Trade [%]                        1.47955
Worst Trade [%]                     -1.389335
Avg. Trade [%]                    

In [62]:
eng.plot()

C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2463', ...)